<a href="https://colab.research.google.com/github/Vaibhavs10/notebooks/blob/main/CodeGemma_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CodeGemma 2B & 7B

CodeGemma is a family of code-specialist LLM models by Google, based on the pre-trained 2B and 7B Gemma checkpoints.

They were trained on top of the base Gemma 2B and 7B models using a mixture of 500 billion tokens of primarily English language data, mathematics, and code.

They leverage the natural language capabilities of their ancestors, improve on logical and mathematical reasoning, and are suitable for code completion and generation.

*Note: make sure, you're on a GPU run-time with Colab.*

## Setup the inference environment

In [ ]:
!pip install -q --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 67.7 MB/s eta 0:00:00


## Load the tokenizer and the model

CodeGemma 2B was trained exclusively on the Code Infilling task and is meant for fast code completion and generation, especially in settings where latency and/or privacy are crucial.

CodeGemma 7B training mix includes code infilling data (80%) and natural language. It can be used for code completion, as well as code and language understanding and generation.

CodeGemma 7B Instruct, in fact, was fine-tuned for instruction following on top of CodeGemma 7B. It’s meant for conversational use, especially around code, programming, or mathematical reasoning topics. It’s not as powerful as the other versions for code completion.

In [ ]:
# pip install accelerate
from transformers import GemmaTokenizer, AutoModelForCausalLM

tokenizer = GemmaTokenizer.from_pretrained("google/codegemma-7b-it")
model = AutoModelForCausalLM.from_pretrained("google/codegemma-7b-it")

input_text = "Write a Python function to calculate the nth fibonacci number.\n"
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
from transformers import GemmaTokenizer, AutoModelForCausalLM
import torch

model_id = "gg-hf/codegemma-2b"

tokenizer = GemmaTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
	torch_dtype=torch.float16
).to("cuda:0")

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## Prompt for Code Infilling i.e. Fill in the Middle.

Code completion can be used for infilling inside code editors. CodeGemma was trained for this task using the fill-in-the-middle (FIM) objective, where you provide a prefix and a suffix as context for the completion. The following tokens are used to separate the different parts of the input:

 - `<|fim_prefix|>` precedes the context before the completion we want to run.
- `<|fim_suffix|>` precedes the suffix. You must put this token exactly where the cursor would be positioned in an editor, as this is the location that will be completed by the model.
- `<|fim_middle|>` is the prompt that invites the model to run the generation.

 In addition to these, there's also `<|file_separator|>`, which is used to provide multi-file contexts.

 Please, make sure to not provide any extra spaces or newlines around the tokens, other than those that would naturally occur in the code fragment you want to complete. Here's an example:


In [ ]:
prompt = '''\
<|fim_prefix|>import datetime
def calculate_age(birth_year):
    """Calculates a person's age based on their birth year."""
    current_year = datetime.date.today().year
    <|fim_suffix|>
    return age<|fim_middle|>\
'''

## Generate code

We tokenize the inputs and pass them through model.generate. You can pass a wide variety of arguments and strategies, read more about them [here](https://huggingface.co/docs/transformers/generation_strategies).

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
prompt_len = inputs["input_ids"].shape[-1]
outputs = model.generate(**inputs, max_new_tokens=100)

In [ ]:
print(tokenizer.decode(outputs[0][prompt_len:]))

age = current_year - birth_year<|file_separator|><eos>


Let's print all of it together?

In [ ]:
print(tokenizer.decode(outputs[0]))

<bos><|fim_prefix|>import datetime
def calculate_age(birth_year):
    """Calculates a person's age based on their birth year."""
    current_year = datetime.date.today().year
    <|fim_suffix|>
    return age<|fim_middle|>age = current_year - birth_year<|file_separator|><eos>


Voila! You have your own personal Code Completion assistant now!